In [14]:
from pathlib import Path
import vectorbt as vbt
from hydra.SuperSim import load_prices

def load_portfolio(name):    
    output_dir = Path.cwd().joinpath('..', 'output')
    filepath = output_dir.joinpath(name)
    return vbt.Portfolio.load(filepath)

vbt.settings.ohlcv["column_names"] = {
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "volume": "volume",
}

pair='XBTUSD'
start='2018-05-15 00:00:00'
end='2018-05-15 1:30:00'
interval=1

#portfolio = load_portfolio(f"{pair} {start} {end} {interval} Aroon 100-101.portfolio")
prices = {}
prices[1] = load_prices(pair, '../data/kraken', start, end, 1)
prices[5] = load_prices(pair, '../data/kraken', start, end, 5)
prices[15] = load_prices(pair, '../data/kraken', start, end, 15)
prices[60] = load_prices(pair, '../data/kraken', start, end, 60)
prices[720] = load_prices(pair, '../data/kraken', start, end,720)
prices[1440] = load_prices(pair, '../data/kraken', start, end, 1440)
AROONOSC = vbt.IndicatorFactory.from_talib("AROONOSC")
# printd(help(AROONOSC.run))
aroonosc = AROONOSC.run(
    prices[1]["high"], prices[1]["low"], timeperiod=[100]
)



In [15]:
import plotly.graph_objects as go
import pandas as pd
import hydra.utils as utils
from datetime import datetime, timedelta


candlestick = go.Candlestick(
        x=prices[1440].index,
        open=prices[1440]["open"],
        high=prices[1440]["high"],
        low=prices[1440]["low"],
        close=prices[1440]["close"],
        hoverinfo="all",
    )
print(prices[1440])

def parsedatetime(dt):
    return pd.to_datetime(dt)
    # return datetime.strptime(dt, '%Y-%m-%d %H:%M')
start='2018-05-15 00:30:00'
end='2018-05-15 1:00:00'

start_date = parsedatetime(start)
end_date = parsedatetime(end)
delta = end_date - start_date
delta_m = delta.total_seconds() / 60
if delta_m < 2000:
    interval = 1
elif delta_m < 2000 * 5:
    interval = 5
elif delta_m < 2000 * 15:
    interval = 15
elif delta_m < 2000 * 60:
    interval = 60
elif delta_m < 2000 * 720:
    interval = 720
else:
    interval = 1440
precise_price = prices[interval].loc[start_date:end_date]
orig_price = prices[1440]
the_price = pd.concat([orig_price, precise_price]).sort_index()
f = open('log.txt', "a")
fakestart = start_date - timedelta(minutes=10)
fakeend = end_date + timedelta(minutes=10)
f.write(f"[{utils.now()}] {start} {end} {delta} {delta_m} {interval} \n{precise_price} \n {fakestart} {fakeend} \n{the_price.loc[fakestart:fakeend]}\n")
f.close()


              open    high     low   close       volume
time                                                   
2018-05-15  8661.6  8862.0  8421.0  8462.8  3875.260105


In [21]:
import numpy as np
from numba import jit
from timeit import default_timer as timer, timeit
from collections import deque
import gc


@jit(nopython=True, fastmath=True)
def fast_id(base, entry, exit):
    return entry * base + exit

@jit(nopython=True, fastmath=True)
def fast_id_zip(base, entry, exit):
    return np.column_stack((entry * base + exit, entry, exit))

def slow_id(base, entry, exit):
    return entry * base + exit

def slow_id_zip(base, entry, exit):
    return np.column_stack((entry * base + exit, entry, exit))

@jit(nopython=True, fastmath=True)
def blaze_it(base, entries, exits):
    result = []
    for (exit_id,) in exits:
        for entry_time, entry_id in entries:
            result.append(fast_id(base, entry_id, exit_id))
    return result


entry_set = deque([(i, i) for i in range(5000)])
exit_set = ([(i,) for i in range(100)])
base = max(len(entry_set), len(exit_set))


def og():
    res = []
    for (exit_id,) in exit_set:
        for entry_time, entry_id in entry_set:
            sim_id = slow_id(base, entry_id, exit_id)
            res = sim_id + entry_id + exit_id
    return

def og_idx():
    res = []
    for (exit_id,) in exit_set:
        for entry in entry_set:
            entry_id = entry[1]
            sim_id = slow_id(base, entry_id, exit_id)
            res = sim_id + entry_id + exit_id
    return

def og_kush():
    res = []
    for (exit_id,) in exit_set:
        for entry_time, entry_id in entry_set:
            sim_id = fast_id(base, entry_id, exit_id)
            res = sim_id + entry_id + exit_id
    return

def dank_og_kush():
    return blaze_it(base, list(entry_set), exit_set)

def numpy_slow():
    entries = np.array(entry_set)
    exits = np.array(exit_set)
    for exit_id in exits:
        for entry_id in entries:
            sim_id = slow_id(base, entry_id, exit_id)
            res = sim_id + entry_id + exit_id
    return
    

def numpy_fast():
    entries = np.array(entry_set)
    exits = np.array(exit_set)
    for exit_id in exits:
        for entry_id in entries:
            sim_id = fast_id(base, entry_id, exit_id)
            res = sim_id + entry_id + exit_id
    return

def fast_enum_idx():
    entries = np.array(entry_set)
    exits = np.array(exit_set)
    combos = np.array(np.meshgrid(entries, exits, indexing='xy')).reshape(2, -1)
    res = fast_id(base, combos[0], combos[1])
    idx = 0
    for sim_id in res:
        entry_id = combos[0][idx]
        exit_id = combos[1][idx]
        idx +=1
        # keyed_exits
        # sim = simulations.get/create_sim/whatever
        res = sim_id + entry_id + exit_id
    return 

def fast_enum():
    entries = np.array(entry_set)
    exits = np.array(exit_set)
    combos = np.array(np.meshgrid(entries, exits, indexing='xy')).reshape(2, -1)
    res = fast_id(base, combos[0], combos[1])
    for idx, sim_id in enumerate(res):
        entry_id = combos[0][idx]
        exit_id = combos[1][idx]
        # keyed_exits
        # sim = simulations.get/create_sim/whatever
        res = sim_id + entry_id + exit_id
    return 


def slow():
    entries = np.array(entry_set)
    exits = np.array(exit_set)
    combos = np.array(np.meshgrid(entries, exits, indexing='xy')).reshape(2, -1)
    return slow_id_zip(base, combos[0], combos[1])


def timr(fn, return_tuple=False):
    ts = timer()
    result = fn()
    te = timer()

    elapsed = te - ts
    # print("func:%r args:[%r, %r] took: %2.4f sec" % (f.__name__, args, kw, te - ts))
    if return_tuple == True:
        return result, elapsed

    print("func:%r took: %2.4f sec" % (fn.__name__, elapsed))
    return result

loops = 50
tests = [
    ('og()', og),
    ('og_idx()', og_idx),
    # ('og_kush()', og_kush),
    # ('dank_og_kush()', dank_og_kush),
    # ('numpy_slow()', numpy_fast),
    # ('numpy_fast()', numpy_fast),
    # ('fast_enum_idx()', fast_enum_idx),
    # ('fast_enum()', fast_enum),
]
counter = {}

print(f"{len(entry_set)=} {len(exit_set)=} {loops=}")

for name, fn in tests:
    print(name, timeit(name, globals=globals(), number=loops))    
    counter[name] = 0

print("=~=~=~=")

for x in range(loops):
    for name, fn in tests:
        _, time = timr(lambda: fn(), return_tuple = True)
        counter[name] += time

for key, val in counter.items():
    print(key, val/loops)


len(entry_set)=5000 len(exit_set)=100 loops=50
og() 5.168998299999657
og_idx() 5.774586399998952
=~=~=~=
og() 0.11068111000015052
og_idx() 0.11803346999997302


In [43]:
from operator import itemgetter

profits = [
    [5, 1, 4],
    [5, 4, 2],
    [4, 7, 3],
    [4, 9, 1],
    [4, 4, 7],
    [3, 2, 5],
    [1, 1, 0]
]
nonlocal maxi = 0
def test_me(profit):
    if profit[0] > 1:
        if profit[1] > maxi:
            maxi = profit[1]
        return True
    return False

test_me((2,4))
#p = [profit for profit in profits if test_me(profit, maxi) ]
#, maxi
# profits = sorted(profits, key = itemgetter(2), reverse=True)
# profits

UnboundLocalError: local variable 'maxi' referenced before assignment